In [0]:
%run /Workspace/Users/vishal.krishnan@neudesic.com/Silver_to_Gold/utils

In [0]:
from pyspark.sql.functions import lit, col, to_date, count, sum, when, countDistinct, expr

# fact_engagement_metrics
engagement_df = df_youtube.select("campaign_id", "date", "impressions", "reach", "clicks", "link_clicks", "conversions", "likes", "shares", "comments") \
    .withColumn("platform_id", lit(103)) \
    .withColumnRenamed("date", "engagement_date")

meta_engage = df_meta.select("campaign_id", "date", "impressions", "reach", "clicks", "link_clicks", "likes", "comments", "shares", "conversions", "ad_spend", "platform_name") \
    .withColumn("platform_id", expr("case when lower(platform_name) = 'facebook' then 101 when lower(platform_name) = 'instagram' then 102 else null end")) \
    .withColumnRenamed("date", "engagement_date")

fact_engagement = engagement_df.unionByName(meta_engage.select("campaign_id", "engagement_date", "impressions", "reach", "clicks", "link_clicks", "likes", "shares", "comments", "conversions", "platform_id"))
fact_engagement = fact_engagement.join(df_web.groupBy("campaign_id", to_date("session_start").alias("engagement_date")) \
    .agg(
        count("session_id").alias("sessions"),
        sum(when(col("bounce") == True, 1).otherwise(0)).alias("bounces"),
        countDistinct("session_id").alias("unique_users"),
        sum("revenue").alias("total_ad_spend")
    ), on=["campaign_id", "engagement_date"], how="left")

fact_engagement = fact_engagement.fillna(0)

display(fact_engagement)

upsert_to_delta(fact_engagement, "fact_engagement_metrics", ["campaign_id", "platform_id", "engagement_date"],
                zorder_cols=["campaign_id", "platform_id", "engagement_date"])